Recently I participated in recommender system contest, where I managed to place among the top 10. In this notebook I provide the code and comparisons of my Item Based K Nearest Neighbours with some other algorithms.

In [1]:
import pandas as pd
import numpy as np
from evaluate_algorithm import inter_matr_implicit
import subprocess

from item_knn import recTopK
from recommenders_to_compare.random_recommender import random_recommender
from recommenders_to_compare.pop_recommender import pop_recommender
from recommenders_to_compare.svd_recommender import svd_decompose, svd_recommend_to_list
from recommenders_to_compare.cb_item_knn import cb_itemknn_recommendation

In [3]:
top_k = 10

# Reading data
def read(dataset, file):
    return pd.read_csv('challenge_data/' + dataset + '.' + file, sep='\t')

def write_recommendations(recommendations_to_write):
    with open('recommendations.tsv', 'w') as f:
        for user_id, item_ids in zip(inters_test['user_id'].unique(), recommendations_to_write):
            line = f"{user_id}\t{','.join(item_ids.astype(str))}\n"
            f.write(line)

def get_accuracy(title, recommendations_to_evaluate):
    write_recommendations(recommendations_to_evaluate)

    result = subprocess.run(["python3", "evaluate_algorithm.py", "--submission", "./recommendations.tsv", "--target", "./challenge_data/lfm-challenge.inter_test"],
                capture_output=True,
                text=True)
    accuracy = float(result.stdout.split("\n")[1].split(" ")[-1][:6])

    print(f"{title}: {round(accuracy * 100, 2)}%")

In [4]:
items = read("lfm-challenge", 'item')
users = read("lfm-challenge", 'user')
item_embeddings = read("lfm-challenge", 'musicnn')
train_inters = read("lfm-challenge", 'inter_train')
inters_test = read("lfm-challenge", 'inter_test')

# Split train_inters into train and validation sets
train_inter_matrix = inter_matr_implicit(users=train_inters['user_id'].max()+1, items=train_inters['item_id'].max()+1, interactions=train_inters)
test_inter_matrix = inter_matr_implicit(users=inters_test['user_id'].max()+1, items=inters_test['item_id'].max()+1, interactions=inters_test)

To have an understanding of how complex the task is, let's look at the performance of random recommender

In [5]:
recommendations = random_recommender(user_ids=inters_test['user_id'].unique(), val_inter_matrix=test_inter_matrix, top_k=top_k)
get_accuracy("Random recommender", recommendations)

Random recommender: 0.25%


Now jumping directly to my recommender

In [6]:
recommendations = []
recommendations_unsorted = recTopK(train_inter_matrix, top_k=top_k, n=2)
for user_id in inters_test['user_id'].unique():
    recommendations.append(recommendations_unsorted[user_id])
recommendations = np.array(recommendations)
get_accuracy("Item based k-nearest neighbours", recommendations)

Computing recommendations: 100%|██████████| 2795/2795 [00:57<00:00, 48.28it/s]


Item based k-nearest neighbours: 20.37%


Not bad. Let's compare this to other approaches

#### Recommender of the most popular songs

In [7]:
recommendations = pop_recommender(user_ids=train_inters['user_id'].unique(), inter_matrix=train_inter_matrix, top_k=top_k)
get_accuracy("POP-recommender", recommendations)

POP-recommender: 0.82%


Single Value Decomposition (SVD) recommender

In [8]:
U, V = svd_decompose(train_inter_matrix, f = 300)
recommendations = []
for user_id in inters_test['user_id'].unique():
    seen_item_ids = train_inters[train_inters["user_id"] == user_id]['item_id']
    recommendations.append(svd_recommend_to_list(user_id=user_id, seen_item_ids=seen_item_ids, U=U, V=V, topK=top_k)[0])
get_accuracy("SVD-recommender", recommendations)

SVD-recommender: 17.07%


Content based Item KNN recommender

In [9]:
recommendations = []
for user_id in inters_test['user_id'].unique():
    seen_item_ids = train_inters[train_inters["user_id"] == user_id]['item_id']
    recommendations.append(cb_itemknn_recommendation(seen_item_ids, item_embeddings, top_k=top_k, knn_k=10))
get_accuracy("CB-recommender", recommendations)

CB-recommender: 1.52%


#### Further investigation

I am trying to visualize recommendation strategies of SVD, ItemKNN, ContentKNN + POP recommender,
maybe we can derive some patterns. Firstly I wanted to make graphs,
but then I realized that, because of data complexity it will be hard to visualize.
Instead, I am eager to calculate Levenshtein, Jaccard, longest common sequence similarities for recommendation for respective users.

##### Levenshtein similarity matrix (counting the minimum number of single-character edits):

                        cknn|0.0152  user_knn|0.2037  pop|0.0082  svd|0.1707
    cknn|0.0152         1.000000         0.017818    0.002147    0.015063
    item_knn|0.2037     0.017818         1.000000    0.004830    0.227048
    pop|0.0082          0.002147         0.004830    1.000000    0.009445
    svd|0.1707          0.015063         0.227048    0.009445    1.000000

##### Jaccard similarity matrix (elements intersection/union):

                        cknn|0.0152  user_knn|0.2037  pop|0.0082  svd|0.1707
    cknn|0.0152         1.000000         0.010365    0.001130    0.008488
    item_knn|0.2037     0.010365         1.000000    0.002637    0.217995
    pop|0.0082          0.001130         0.002637    1.000000    0.005045
    svd|0.1707          0.008488         0.217995    0.005045    1.000000

##### LCS similarity matrix (longest common subsequence):

                        cknn|0.0152  user_knn|0.2037  pop|0.0082  svd|0.1707
    cknn|0.0152        10.000000         0.178175    0.021467    0.150626
    item_knn|0.2037     0.178175        10.000000    0.048301    2.270483
    pop|0.0082          0.021467         0.048301   10.000000    0.094454
    svd|0.1707          0.150626         2.270483    0.094454   10.000000

we can observe that user_knn and SVD have relatively strong levels of similarity, so, SVD and Item_KNN probably recommend similar items in different order

### Conclusion
After my best system was a plain ItemKNN. On the competition platform I achieved 38.3% accuracy calculating recommendations from the whole dataset. This is a quite good result. It could be better, if I had an inside about time factor of listens, meaning a time sequence of listens. Further improvements can be achieved being provided with more information about user.